In [36]:
import torch
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torchvision import datasets, transforms
from tqdm import tqdm
from sklearn.decomposition import PCA

In [2]:
import wandb

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sanika-damle (project-ai-scream). Use `wandb login --relogin` to force relogin


True

In [5]:
test = datasets.CIFAR10(root='/scratch/cifar10', train=False, download=True, transform=ToTensor())

100%|██████████| 170498071/170498071 [09:45<00:00, 291175.94it/s] 


Extracting /scratch/cifar10/cifar-10-python.tar.gz to /scratch/cifar10


In [6]:
train = datasets.CIFAR10(root='/scratch/cifar10', train=True, download=True, transform=ToTensor())

Files already downloaded and verified


In [8]:
loader = {
    'train': DataLoader(train, batch_size=256, shuffle=True),
    'test': DataLoader(test, batch_size=512, shuffle=True)
}

In [65]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [45]:
class VGG16(nn.Module):
    def __init__(self, num_classes, batchnorm):
        super(VGG16, self).__init__()
        self.layer1 = self._make_layer(3, 64, batchnorm)
        self.layer2 = self._make_layer(64, 64, batchnorm, maxpool=True)
        self.layer3 = self._make_layer(64, 128, batchnorm)
        self.layer4 = self._make_layer(128, 128, batchnorm, maxpool=True)
        self.layer5 = self._make_layer(128, 256, batchnorm)
        self.layer6 = self._make_layer(256, 256, batchnorm)
        self.layer7 = self._make_layer(256, 256, batchnorm, maxpool=True)
        self.layer8 = self._make_layer(256, 512, batchnorm)
        self.layer9 = self._make_layer(512, 512, batchnorm)
        self.layer10 = self._make_layer(512, 512, batchnorm, maxpool=True)
        self.layer11 = self._make_layer(512, 512, batchnorm)
        self.layer12 = self._make_layer(512, 512, batchnorm)
        self.layer13 = self._make_layer(512, 512, batchnorm, maxpool=True)
        self.layer14 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 4096),
            nn.ReLU(),
        )
        self.layer15 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
        )
        self.layer16 = nn.Sequential(
            nn.Linear(4096, num_classes),
        )

    def _make_layer(self, in_channels, out_channels, batchnorm, maxpool=False):
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1),
        ]
        
        if batchnorm:
            layers.append(nn.BatchNorm2d(out_channels))
            
        layers.append(nn.ReLU())
        
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.view(out.size(0), -1)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        return out
   
    def fit(self, loaders, epochs, device, learning_rate):
        self.to(device)
        criterion = nn.CrossEntropyLoss().to(device)
        # adamoptimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        for epoch in tqdm(range(epochs)):
            self.train().to(device)

            total_loss = 0
            correct = 0
            total = 0
            initial_weights5 = self.layer5[0].weight.data.clone().detach()
            initial_weights10 = self.layer10[0].weight.data.clone().detach()

            for i, (images, labels) in enumerate(loaders['train']):
                self.train().to(device)
                optimizer.zero_grad()

                images = images.to(device)
                labels = labels.to(device)
                
                outputs = self(images)
                loss = criterion(outputs, labels)
                
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                if(i % 30 == 0):
                    wandb.log({"Epoch": epoch + 1, "Loss": total_loss / (i + 1), "Train_Accuracy": 100 *correct / total})

                if(i%100 == 0):
                    final_weights5 = self.layer5[0].weight.data.clone().detach()
                    final_weights10 = self.layer10[0].weight.data.clone().detach()
                    cos5 = self.cosine_similarity(initial_weights5, final_weights5)
                    cos10 = self.cosine_similarity(initial_weights10, final_weights10) 
                    wandb.log({"Epoch": epoch + 1, "Cosine Similarity Layer 5": cos5})
                    wandb.log({"Epoch": epoch + 1, "Cosine Similarity Layer 10": cos10}) 
                    l2_norm5 = torch.norm(final_weights5 - initial_weights5, p=2)
                    l2_norm10 = torch.norm(final_weights10 - initial_weights10, p=2)
                    wandb.log({"Epoch": epoch + 1, "L2 Layer 5": l2_norm5})
                    wandb.log({"Epoch": epoch + 1, "L2 Layer 10": l2_norm10})   
                    
            # total_loss = total_loss / len(loaders['train'])
            # accuracy = 100 * correct / total
            # print('Epoch: {}, Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, total_loss, accuracy))
            # get final weights for layer 5
            # calculate cosine similarity